In [4]:
%%writefile model_gin.py

from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Model, Sequential
import numpy as np
import tensorflow as tf
import gin

@gin.configurable
class BiLSTMGRUSpatialDropout1D():
    def __init__(self, nb_words, embedding_size, embedding_matrix=None, is_embedding_trainable=False, h_lstm=256, h_gru=128):
        
        if embedding_matrix is None:
            embedding_matrix = np.zeros((nb_words, embedding_size))

        self.nb_words = nb_words
        self.embedding_size = embedding_size
        self.embedding_matrix = embedding_matrix
        self.is_embedding_trainable = is_embedding_trainable
        self.h_lstm = h_lstm
        self.h_gru = h_gru
        
    def __call__(self,pre_layer):
        x = Embedding(self.nb_words, self.embedding_size, weights=[self.embedding_matrix], trainable=self.is_embedding_trainable)(pre_layer)
        x = SpatialDropout1D(0.3)(x)
        x1 = Bidirectional(LSTM(self.h_lstm, return_sequences=True))(x)
        x2 = Bidirectional(GRU(self.h_gru, return_sequences=True))(x1)
        max_pool1 = GlobalMaxPooling1D()(x1)
        max_pool2 = GlobalMaxPooling1D()(x2)
        conc = Concatenate()([max_pool1, max_pool2])
        return conc


Overwriting model_gin.py


In [15]:
%%writefile config.gin
import model_gin

BiLSTMGRUSpatialDropout1D.nb_words = 100
BiLSTMGRUSpatialDropout1D.embedding_size = 1000


Overwriting config.gin


## Test

In [16]:
%%writefile test.py

import gin
gin.parse_config_file('config.gin')

from model_gin import BiLSTMGRUSpatialDropout1D
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

input_shape = (100,)
model_input = Input(shape=input_shape)
bilstm_layers = BiLSTMGRUSpatialDropout1D()(model_input)

output = Dense(3, activation='softmax')(bilstm_layers)
full_model = Model(inputs=model_input, outputs=output)
print(full_model.summary())

Overwriting test.py


In [14]:
!python test.py

2020-04-19 16:44:04.457103: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-19 16:44:04.466669: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fde42fbea30 executing computations on platform Host. Devices:
2020-04-19 16:44:04.466699: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 1000)    10000       input_1[0][0]                    
________________________________

In [17]:
!python test.py

2020-04-19 16:44:46.961324: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-19 16:44:46.972030: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fec6c6e9ec0 executing computations on platform Host. Devices:
2020-04-19 16:44:46.972047: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 1000)    100000      input_1[0][0]                    
________________________________

## Normal Way of Model import

In [1]:
import model_gin

In [2]:
from model_gin import BiLSTMGRUSpatialDropout1D
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

input_shape = (100,)
model_input = Input(shape=input_shape)
bilstm_layers = BiLSTMGRUSpatialDropout1D(10,1000)(model_input)

output = Dense(3, activation='softmax')(bilstm_layers)
full_model = Model(inputs=model_input, outputs=output)
print(full_model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 1000)    10000       input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 1000)    0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 512)     2574336     spatial_dropout1d[0][0]          
______________________________________________________________________________________________